# Challenge Data Analysis

## Import modules

In [1]:
import numpy as np
import pandas as pd

## Files used

In [2]:
filename_in = './data/immoweb_scrapped.csv'
filename_out = './data/immoweb_cleaned.csv'

## Load data

In [3]:
data = pd.read_csv(filename_in, index_col="id")
data.head()

locality type_of_property subtype_of_property      price  \
id                                                                  
9044081      1083        APARTMENT           APARTMENT   265000.0   
9043978      1000        APARTMENT           APARTMENT  1795000.0   
9044188      1050            HOUSE             MANSION  3800000.0   
9041095      4860            HOUSE               HOUSE   320000.0   
9042175      1160  APARTMENT_GROUP     APARTMENT_GROUP        NaN   

        type_of_sale  nr_of_rooms   area     equiped_kitchen furnished  \
id                                                                       
9044081     FOR_SALE          4.0   90.0           INSTALLED     False   
9043978     FOR_SALE          4.0  650.0  USA_HYPER_EQUIPPED     False   
9044188     FOR_SALE          5.0  752.0      HYPER_EQUIPPED     False   
9041095     FOR_SALE          5.0  231.0       NOT_INSTALLED     False   
9042175     FOR_SALE          NaN    NaN                 NaN       NaN   

         open_fire terrace  terrace_area garden  garden_area  total_land_area  \
id                                                                              
9044081      False    True          13.0    NaN          NaN              NaN   
9043978       True    True         400.0    NaN          NaN              NaN   
9044188      False    True          40.0   True          NaN            340.0   
9041095      False    True          30.0   True       1200.0           1421.0   
9042175      False     NaN           NaN    NaN          NaN              NaN   

         nr_of_facades swimming_pool building_condition  
id                                                       
9044081            4.0           NaN             AS_NEW  
9043978            3.0           NaN             AS_NEW  
9044188            2.0           NaN     JUST_RENOVATED  
9041095            3.0         False             AS_NEW  
9042175            NaN           NaN                NaN

## Initial check
Which values are missing?

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10006 entries, 9044081 to 8471905
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   locality             10006 non-null  int64  
 1   type_of_property     10006 non-null  object 
 2   subtype_of_property  10006 non-null  object 
 3   price                9054 non-null   float64
 4   type_of_sale         10006 non-null  object 
 5   nr_of_rooms          9119 non-null   float64
 6   area                 8817 non-null   float64
 7   equiped_kitchen      7819 non-null   object 
 8   furnished            5535 non-null   object 
 9   open_fire            10006 non-null  bool   
 10  terrace              6421 non-null   object 
 11  terrace_area         4373 non-null   float64
 12  garden               3029 non-null   object 
 13  garden_area          2097 non-null   float64
 14  total_land_area      4576 non-null   float64
 15  nr_of_facades        7024 no

## Clean data

### No duplicates

In [5]:
data.index.is_unique

True

### No blank space

In [7]:
# Remove leading and trailing withespaces
def strip_str(x):
    if type(x) is str:
        return x.strip()
    return x

In [8]:
data = data.applymap(strip_str)

In [9]:
data.head()

locality type_of_property subtype_of_property      price  \
id                                                                  
9044081      1083        APARTMENT           APARTMENT   265000.0   
9043978      1000        APARTMENT           APARTMENT  1795000.0   
9044188      1050            HOUSE             MANSION  3800000.0   
9041095      4860            HOUSE               HOUSE   320000.0   
9042175      1160  APARTMENT_GROUP     APARTMENT_GROUP        NaN   

        type_of_sale  nr_of_rooms   area     equiped_kitchen furnished  \
id                                                                       
9044081     FOR_SALE          4.0   90.0           INSTALLED     False   
9043978     FOR_SALE          4.0  650.0  USA_HYPER_EQUIPPED     False   
9044188     FOR_SALE          5.0  752.0      HYPER_EQUIPPED     False   
9041095     FOR_SALE          5.0  231.0       NOT_INSTALLED     False   
9042175     FOR_SALE          NaN    NaN                 NaN       NaN   

         open_fire terrace  terrace_area garden  garden_area  total_land_area  \
id                                                                              
9044081      False    True          13.0    NaN          NaN              NaN   
9043978       True    True         400.0    NaN          NaN              NaN   
9044188      False    True          40.0   True          NaN            340.0   
9041095      False    True          30.0   True       1200.0           1421.0   
9042175      False     NaN           NaN    NaN          NaN              NaN   

         nr_of_facades swimming_pool building_condition  
id                                                       
9044081            4.0           NaN             AS_NEW  
9043978            3.0           NaN             AS_NEW  
9044188            2.0           NaN     JUST_RENOVATED  
9041095            3.0         False             AS_NEW  
9042175            NaN           NaN                NaN

### No errors

##### Price error

In [10]:
data = data[data['price'] != 1]

##### Area error

In [11]:
data.loc[data['area'] == 1, 'area'] = -1

### No empty values

##### Empty price
Removing all the data without a price


In [12]:
data = data.dropna(subset=['price'])

In [13]:
data['price'].min()

20000.0

##### Area
Try to determinate the area on basis of :
- total land area
- garden area

If not possible to determinate it, -1

In [14]:
def area_adj(x):
    if x['area'] != x['area']:
        if x['total_land_area'] > x['garden_area']:
            return x['total_land_area'] - x['garden_area']
        else:
            return -1
    else:
        return x['area']

In [15]:
data['area'] = data.apply(area_adj, axis= 1)

In [16]:
data['area'].value_counts()

-1.0      272
 100.0    184
 120.0    172
 90.0     168
 110.0    155
         ... 
 578.0      1
 568.0      1
 407.0      1
 586.0      1
 349.0      1
Name: area, Length: 551, dtype: int64

##### Furnished

In [17]:
data['furnished'] = data['furnished'].fillna(False)

In [18]:
data['furnished'].value_counts()

False    8654
True      399
Name: furnished, dtype: int64

##### Terrace


In [19]:
data['terrace'] = data['terrace'].fillna(False)

In [20]:
data['terrace'].value_counts()

True     6392
False    2661
Name: terrace, dtype: int64

##### Terrace area

In [21]:
data['terrace_area'] = data['terrace_area'].fillna(-1)

In [22]:
data['terrace_area'].value_counts()

-1.0      4689
 20.0      268
 10.0      231
 15.0      216
 30.0      196
          ... 
 84.0        1
 104.0       1
 165.0       1
 89.0        1
 106.0       1
Name: terrace_area, Length: 152, dtype: int64

##### Garden
Try to determinate the presence on basis of :

- total land area
- terrace_area
- area

In [23]:
# Function that will replace the NaN garden value 
# by a boolean (True/False)
# True if total_land_area > (area + terrace_area)
# otherwise False
def garden_nan_to_bool_value(x):
    # Only if value is NaN
    if x["garden"] != x["garden"]:
        if x["total_land_area"] > x["area"] + x["terrace_area"]:
            return True
        else:
            return False
    return x["garden"]

In [24]:
data["garden"] = data.apply(garden_nan_to_bool_value, axis=1)

In [25]:
data['garden'].value_counts()

False    4783
True     4270
Name: garden, dtype: int64

##### Total land area
Try to determinate the garden area on basis of :

- terrace_area
- area

If not possible to determinate it, -1

In [26]:
def total_land_area_nan_to_value(x):
    # Only if value is NaN
    if x["total_land_area"] != x["total_land_area"]:
        return x["area"] + x["terrace_area"]
    return x["total_land_area"]

In [27]:
data["total_land_area"] = data.apply(total_land_area_nan_to_value, axis=1)

##### Garden area
Try to determinate the garden area on basis of :

- total land area
- terrace_area
- area

If not possible to determinate it, -1

In [28]:
# Function that will replace the NaN garden_are value 
# by a int
# if total_land_area > (area + terrace_area)
# otherwise 0
def garden_area_nan_to_value(x):
    # Only if value is NaN
    if x["garden_area"] != x["garden_area"]:
        if x["total_land_area"] >= x["area"] + x["terrace_area"]:
            return x["total_land_area"] - x["area"] - x["terrace_area"]
        else:
            return -1
    return x["garden_area"]

In [29]:
data["garden_area"] = data.apply(garden_area_nan_to_value, axis=1)

In [30]:
data['garden_area'].value_counts()

 0.0        4275
-1.0         791
 100.0        62
 200.0        54
 1.0          49
            ... 
 1169.0        1
 4278.0        1
 820.0         1
 15844.0       1
 1287.0        1
Name: garden_area, Length: 1343, dtype: int64

##### Swimming pool

In [31]:
data['swimming_pool'] = data['swimming_pool'].fillna(False)

##### Number of facades

In [32]:
data['nr_of_facades'] = data['nr_of_facades'].fillna(-1)

##### Building condition

In [33]:
data['building_condition'] = data['building_condition'].fillna('UKN')

### Added values

##### Kitchen

In [34]:
#replace all nan values by zero (0) in the column
data['equiped_kitchen'] = data['equiped_kitchen'].replace(np.nan, 'UNK')

#create a new column with condition over 'equiped_kitchen' column
l = ['UNK','NOT_INSTALLED', 'USA_UNINSTALLED']

data['kitchen'] = np.where(data['equiped_kitchen'].isin(l), True, False )

In [35]:
data['kitchen'].value_counts()

False    7327
True     1726
Name: kitchen, dtype: int64

In [36]:
data['equiped_kitchen'].value_counts()

INSTALLED             3357
HYPER_EQUIPPED        1880
UNK                   1295
SEMI_EQUIPPED          969
USA_HYPER_EQUIPPED     839
NOT_INSTALLED          421
USA_INSTALLED          206
USA_SEMI_EQUIPPED       76
USA_UNINSTALLED         10
Name: equiped_kitchen, dtype: int64

##### Region and Province

In [37]:
# Function that will determine the Region
# based on the Locality ZipCode
# 
#1000–1299 -> BXL
#1300–1499 -> WAL
#1500–3999 -> VLA
#4000–4999 -> WAL
#5000–7999 -> WAL
#8000–9999 -> VLA
def get_region_on_zip_code(x):
    # Only if value is NaN
    if (1000 <= x <= 1299):
        return "BXL"
    elif (1300 <= x <= 1499) or (4000 <= x <= 7999):
        return "WAL"
    else:
        return "VLA"

In [38]:
data["region"] = data.locality.apply(get_region_on_zip_code)

In [39]:
data["region"].value_counts()

VLA    3954
WAL    3052
BXL    2047
Name: region, dtype: int64

In [40]:
# function to recognize the province based on the postal code
def province(x):
    prov = []
    for i in x["locality"]:
        if 2000 <= i <= 2999:  # Antwerp
            prov.append("VAN")
        if 3500 <= i <= 3999:  # Limburg
            prov.append("VLI")
        if 4000 <= i <= 4999:  # Liège
            prov.append("WLG")
        if 1300 <= i <= 1499:  # Brabant Wallon
            prov.append("WBR")
        if 5000 <= i <= 5999:  # Namur
            prov.append("WNA") 
        if 6600 <= i <= 6999:  # Luxembourg
            prov.append("WLX")            
        if 1000 <= i <= 1299:  # Brussels
            prov.append("BXL")
        if (1500 <= i <= 1999) | (3000 <= i <= 3499):  # Brabant Flamand
            prov.append("VBR")
        if (6000 <= i <= 6599) | (7000 <= i <= 7999):  # Hainaut
            prov.append("WHT")            
        if 8000 <= i <= 8999:  # Flander Occidentale
            prov.append("VWV") 
        if 9000 <= i <= 9999:  # Flander Orientale
            prov.append("VOV")
    return prov

In [41]:
data["province"] = province(data)

In [42]:
data["province"].value_counts()

BXL    2047
VWV    1739
WLG    1101
VBR     800
VOV     781
WHT     702
WBR     564
VAN     550
WLX     389
WNA     296
VLI      84
Name: province, dtype: int64

##### Price meter price
On the build area and the total land area

In [43]:
# Price / Area 
def price_sq_meter(x):
    if x["area"] > 0:
        return x["price"]/x["area"]
    return -1

In [44]:
data["sq_m_price"] = data.apply(price_sq_meter, axis=1)

In [45]:
data["sq_m_price"].value_counts()

-1.000000       272
 2500.000000     56
 3000.000000     40
 2000.000000     32
 3750.000000     24
               ... 
 4447.761194      1
 2959.259259      1
 3625.541126      1
 1091.205212      1
 2251.184834      1
Name: sq_m_price, Length: 5598, dtype: int64

In [46]:
# Price / Total land area
def price_sq_meter_land(x):
    if x["total_land_area"] > 0:
        return x["price"]/x["total_land_area"]
    return -1

In [47]:
data["sq_m_land_price"] = data.apply(price_sq_meter_land, axis=1)

In [48]:
data["sq_m_land_price"].value_counts()

-1.000000       297
 2500.000000     40
 3000.000000     20
 5000.000000     15
 1000.000000     15
               ... 
 776.315789       1
 7968.085106      1
 4303.030303      1
 2366.336634      1
 605.555556       1
Name: sq_m_land_price, Length: 6820, dtype: int64

## Conversion


##### bool -> 0/1

In [49]:
def bool_to_int(x):
    if type(x) is bool:
        if x:
            return 1
        else:
            return 0
    return x

In [50]:
data = data.applymap(bool_to_int)

In [51]:
data.head()

locality type_of_property subtype_of_property      price  \
id                                                                  
9044081      1083        APARTMENT           APARTMENT   265000.0   
9043978      1000        APARTMENT           APARTMENT  1795000.0   
9044188      1050            HOUSE             MANSION  3800000.0   
9041095      4860            HOUSE               HOUSE   320000.0   
9043036      9600        APARTMENT           APARTMENT   195000.0   

        type_of_sale  nr_of_rooms   area     equiped_kitchen  furnished  \
id                                                                        
9044081     FOR_SALE          4.0   90.0           INSTALLED          0   
9043978     FOR_SALE          4.0  650.0  USA_HYPER_EQUIPPED          0   
9044188     FOR_SALE          5.0  752.0      HYPER_EQUIPPED          0   
9041095     FOR_SALE          5.0  231.0       NOT_INSTALLED          0   
9043036     FOR_SALE          2.0   75.0           INSTALLED          0   

         open_fire  ...  garden_area  total_land_area  nr_of_facades  \
id                  ...                                                
9044081          0  ...          0.0            103.0            4.0   
9043978          1  ...          0.0           1050.0            3.0   
9044188          0  ...         -1.0            340.0            2.0   
9041095          0  ...       1200.0           1421.0            3.0   
9043036          0  ...          0.0             74.0            2.0   

         swimming_pool  building_condition  kitchen  region province  \
id                                                                     
9044081              0              AS_NEW        0     BXL      BXL   
9043978              0              AS_NEW        0     BXL      BXL   
9044188              0      JUST_RENOVATED        0     BXL      BXL   
9041095              0              AS_NEW        1     WAL      WLG   
9043036              0                GOOD        0     VLA      VOV   

          sq_m_price sq_m_land_price  
id                                    
9044081  2944.444444     2572.815534  
9043978  2761.538462     1709.523810  
9044188  5053.191489    11176.470588  
9041095  1385.281385      225.193526  
9043036  2600.000000     2635.135135  

[5 rows x 23 columns]

##### Round float to 2 decimal

In [52]:
def float_round_2_decimal(x):
    if type(x) is float:
        return round(x, 2)
    return x

In [53]:
data = data.applymap(float_round_2_decimal)

##### Convert text dictionary to numeric

In [54]:
dic_type_of_property = {
        "APARTMENT": 1,
        "HOUSE":     2
    }

data["type_of_property"] = data["type_of_property"].apply(lambda x: dic_type_of_property.get(x))

In [55]:
dic_subtype_of_property = {
        "APARTMENT":1,
        "MANSION":2,
        "HOUSE":3,
        "TRIPLEX":4,
        "VILLA":5,
        "FLAT_STUDIO":6,
        "EXCEPTIONAL_PROPERTY":7,
        "LOFT":8,
        "DUPLEX":9,
        "SERVICE_FLAT":10,
        "TOWN_HOUSE":11,
        "FARMHOUSE":12,
        "PENTHOUSE":13,
        "BUNGALOW":14,
        "APARTMENT_BLOCK":15,
        "COUNTRY_COTTAGE":15,
        "MIXED_USE_BUILDING":17,
        "GROUND_FLOOR":17,
        "OTHER_PROPERTY":19,
        "MANOR_HOUSE":20,
        "CHALET":21,
        "KOT":22,
        "CASTLE":23
    }
data["subtype_of_property"] = data["subtype_of_property"].apply(lambda x: dic_subtype_of_property.get(x))

In [56]:
dic_type_of_sale = {"FOR_SALE": 1}
data["type_of_sale"] = data["type_of_sale"].apply(lambda x: dic_type_of_sale.get(x))

In [57]:
dic_equiped_kitchen = {
        "INSTALLED": 1,
        "USA_HYPER_EQUIPPED": 2,
        "HYPER_EQUIPPED": 3,
        "NOT_INSTALLED": 4,
        "SEMI_EQUIPPED": 5,
        "USA_INSTALLED": 6,
        "USA_SEMI_EQUIPPED": 7,
        "UNK": -1,
        "USA_UNINSTALLED": 8
    }
data["equiped_kitchen"] = data["equiped_kitchen"].apply(lambda x: dic_equiped_kitchen.get(x))

In [58]:
dic_building_condition = {
        "AS_NEW": 1,
        "JUST_RENOVATED": 2,
        "GOOD": 3,
        "TO_BE_DONE_UP": 4,
        "TO_RENOVATE": 5,
        "UKN": -1,
        "TO_RESTORE": 6
    }
data["building_condition"] = data["building_condition"].apply(lambda x: dic_building_condition.get(x))

In [59]:
dic_region = {
        "BXL": 0,
        "VLA": 1,
        "WAL": 2
    }
data["region"] = data["region"].apply(lambda x: dic_region.get(x))

In [60]:
dic_province = {
        "BXL": 1,
        "VAN": 2,
        "VBR": 3,
        "VLI": 4,
        "VOV": 5,
        "VWV": 6,
        "WBR": 7,
        "WHT": 8,
        "WLG": 9,
        "WLX": 10,
        "WNA": 11
    }
data["province"] = data["province"].apply(lambda x: dic_province.get(x))

## Fix dtype

In [61]:
data.dtypes

locality                 int64
type_of_property         int64
subtype_of_property      int64
price                  float64
type_of_sale             int64
nr_of_rooms            float64
area                   float64
equiped_kitchen          int64
furnished                int64
open_fire                int64
terrace                  int64
terrace_area           float64
garden                   int64
garden_area            float64
total_land_area        float64
nr_of_facades          float64
swimming_pool            int64
building_condition       int64
kitchen                  int64
region                   int64
province                 int64
sq_m_price             float64
sq_m_land_price        float64
dtype: object

In [62]:
convert_dict = {'price': int,
                'nr_of_rooms': int,
                'area': int,
                'furnished': int,
                'terrace_area': int,
                'garden_area': int, 
                'nr_of_facades': int,
                'total_land_area': int 
               } 
  
data = data.astype(convert_dict)

In [63]:
data.dtypes

locality                 int64
type_of_property         int64
subtype_of_property      int64
price                    int64
type_of_sale             int64
nr_of_rooms              int64
area                     int64
equiped_kitchen          int64
furnished                int64
open_fire                int64
terrace                  int64
terrace_area             int64
garden                   int64
garden_area              int64
total_land_area          int64
nr_of_facades            int64
swimming_pool            int64
building_condition       int64
kitchen                  int64
region                   int64
province                 int64
sq_m_price             float64
sq_m_land_price        float64
dtype: object

In [64]:
data.shape

(9053, 23)

In [65]:
data.head(100)

locality  type_of_property  subtype_of_property    price  \
id                                                                  
9044081      1083                 1                    1   265000   
9043978      1000                 1                    1  1795000   
9044188      1050                 2                    2  3800000   
9041095      4860                 2                    3   320000   
9043036      9600                 1                    1   195000   
...           ...               ...                  ...      ...   
9043293      8020                 2                    3   495000   
9007296      5060                 1                    9   318000   
9029109      4140                 2                   14   349000   
9029927      5580                 2                    3   599000   
9029376      1050                 2                   17   985000   

         type_of_sale  nr_of_rooms  area  equiped_kitchen  furnished  \
id                                                                     
9044081             1            4    90                1          0   
9043978             1            4   650                2          0   
9044188             1            5   752                3          0   
9041095             1            5   231                4          0   
9043036             1            2    75                1          0   
...               ...          ...   ...              ...        ...   
9043293             1            3   217                3          0   
9007296             1            3   118                3          0   
9029109             1            3   180                5          0   
9029927             1            4   320                1          0   
9029376             1            3   340                4          0   

         open_fire  ...  garden_area  total_land_area  nr_of_facades  \
id                  ...                                                
9044081          0  ...            0              103              4   
9043978          1  ...            0             1050              3   
9044188          0  ...           -1              340              2   
9041095          0  ...         1200             1421              3   
9043036          0  ...            0               74              2   
...            ...  ...          ...              ...            ...   
9043293          1  ...          230             1045              3   
9007296          0  ...            0              140              2   
9029109          0  ...          850             1674              4   
9029927          1  ...         4300             4342              4   
9029376          0  ...          100              210              2   

         swimming_pool  building_condition  kitchen  region  province  \
id                                                                      
9044081              0                   1        0       0         1   
9043978              0                   1        0       0         1   
9044188              0                   2        0       0         1   
9041095              0                   1        1       2         9   
9043036              0                   3        0       1         5   
...                ...                 ...      ...     ...       ...   
9043293              0                   1        0       1         6   
9007296              0                   3        0       2        11   
9029109              0                   1        0       2         9   
9029927              0                   1        0       2        11   
9029376              0                   3        1       0         1   

         sq_m_price  sq_m_land_price  
id                                    
9044081     2944.44          2572.82  
9043978     2761.54          1709.52  
9044188     5053.19         11176.47  
9041095     1385.28           225.19  
9043036     2600.00          2635.14  
...           

### Delete unrelevant rows

##### Area unknown
Area must be > 0

In [66]:
data = data[data["area"] > 0]

##### Apartement block
They are whole building and not just a real estate property 

In [67]:
data = data[data["subtype_of_property"] != 15]

##### Price / SqM unknown
Price / Square Meter must be > 0

In [68]:
data = data[data["sq_m_price"] > 0]

### Delete unrelevant columns

##### Type of sale
Always FOR_SALE

In [73]:
data.drop(["type_of_sale"], axis=1, inplace=True)

##### Furnished

### Final Data check

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8577 entries, 9044081 to 8471905
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   locality             8577 non-null   int64  
 1   type_of_property     8577 non-null   int64  
 2   subtype_of_property  8577 non-null   int64  
 3   price                8577 non-null   int64  
 4   nr_of_rooms          8577 non-null   int64  
 5   area                 8577 non-null   int64  
 6   equiped_kitchen      8577 non-null   int64  
 7   furnished            8577 non-null   int64  
 8   open_fire            8577 non-null   int64  
 9   terrace              8577 non-null   int64  
 10  terrace_area         8577 non-null   int64  
 11  garden               8577 non-null   int64  
 12  garden_area          8577 non-null   int64  
 13  total_land_area      8577 non-null   int64  
 14  nr_of_facades        8577 non-null   int64  
 15  swimming_pool        8577 non

### Save data cleaned

In [70]:
data.to_csv(filename_out)